# Loan Approval Applicant Prediction Data Cleaning

In [2]:
# Import Library
import pandas as pd

In [8]:
# Import Loan Applicant data
df = pd.read_csv('LoanApplicantData.csv')
df.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


**Step 1: Explore the dataset to understand what needs fixing**

In [12]:
# Shape of the dataset
print(f"Rows: {df.shape[0]}, Columns: {df.shape[1]}")

Rows: 614, Columns: 13


In [13]:
# Summary of dataset
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [14]:
# Quick Stats for numeric & Object Columns
print(df.describe(include ='all'))

         Loan_ID Gender Married Dependents Education Self_Employed  \
count        614    601     611        599       614           582   
unique       614      2       2          4         2             2   
top     LP001002   Male     Yes          0  Graduate            No   
freq           1    489     398        345       480           500   
mean         NaN    NaN     NaN        NaN       NaN           NaN   
std          NaN    NaN     NaN        NaN       NaN           NaN   
min          NaN    NaN     NaN        NaN       NaN           NaN   
25%          NaN    NaN     NaN        NaN       NaN           NaN   
50%          NaN    NaN     NaN        NaN       NaN           NaN   
75%          NaN    NaN     NaN        NaN       NaN           NaN   
max          NaN    NaN     NaN        NaN       NaN           NaN   

        ApplicantIncome  CoapplicantIncome  LoanAmount  Loan_Amount_Term  \
count        614.000000         614.000000  592.000000         600.00000   
unique 

**Step 2: Handle Missing Values**

In [16]:
# Count Missing Values
missing = df.isnull().sum()
print(missing)

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64


In [19]:
missing = missing[missing > 0].sort_values(ascending=False)
print(missing)

Credit_History      50
Self_Employed       32
LoanAmount          22
Dependents          15
Loan_Amount_Term    14
Gender              13
Married              3
dtype: int64


**Deal with missing Values for Object & Numeric**

In [20]:
# Fill categorical columns with mode
cat_cols = df.select_dtypes(include='object').columns
for col in cat_cols:
    if df[col].isnull().sum() > 0:
        df[col].fillna(df[col].mode()[0], inplace=True)

# The mode is perfect for majority categories in classification tasks.

In [24]:
# Fill numerical columns with median
num_cols = df.select_dtypes(include=['int64', 'float64']).columns
for col in num_cols:
    if df[col].isnull().sum()>0:
        df[col].fillna(df[col].mode()[0], inplace=True)

# The median is robust to outliers.

**Step 3: Fix Data Types**

This helps with memory optimization and model compatibility.

In [29]:
# Convert 'Loanamount' to float
df['loanAmount'] = df['LoanAmount'].astype(float)

# Convert Categorical Variables to Category Type
for col in cat_cols:
    df[col] = df[col].astype('category')

**Step 4: Standardize Text Columns**

In [30]:
# strip whitespace and convert to consistent case
for col in cat_cols:
    df[col] = df[col].str.strip().str.lower()

**Step 5: Handle Outliers**

In [31]:
# capping outliers at 99th percentile
for col in ['LoanAmount', 'ApplicantIncome']:
    upper = df[col].quantile(0.99)
    df[col] = df[col].apply(lambda x: upper if x > upper else x)

In [32]:
df.to_csv('cleaned_loan_data.csv', index=False)

**Pipeline Function**

In [34]:
def clean_loan_data(df):
    # fill missing values
    for col in df.select_dtypes(include='object'):
        df[col].fillna(df[col].mode()[0], inplace=True)
    for col in df.select_dtypes(include=['int64', 'float64']):
        df[col].fillna(df[col].median(), inplace=True)

    # type conversions
    for col in df.select_dtypes(include='object'):
        df[col] = df[col].astype('category')
        df[col] = df[col].str.strip().str.lower()

    # outlier capping
    for col in ['LoanAmount', 'ApplicantIncome']:
        if col in df.columns:
            upper = df[col].quantile(0.99)
            df[col] = df[col].apply(lambda x: upper if x > upper else x)

    return df

In [35]:
df = pd.read_csv('LoanApplicantData.csv')
df_clean = clean_loan_data(df)